# Test cascade failure dynamic

In [6]:
# Test cascading dynamic on a single food product

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm

In [7]:
# FUNCTIONS
def Trade_preferences (data_trade,unique_ISO): 
    # Define trade matrix
    matrix_trade = data_trade.pivot(index = 'origin_country_ISO',columns='destin_country_ISO',values='value').fillna(0)
    row_sum= matrix_trade.sum(axis=1) #axis =1 is rownorm
    row_sum= row_sum[row_sum>0]

    # Remove rows with zero sum 
    matrix_trade_simp= matrix_trade.loc[row_sum.index,:] 
    df_trade_simp = pd.DataFrame(matrix_trade_simp, columns=matrix_trade_simp.index, index=matrix_trade_simp.columns) 
    inv_sum = np.diag(1/row_sum.to_numpy())

    # Get matrix multiplication
    W = np.matmul(inv_sum,matrix_trade_simp.to_numpy())
    W_df = pd.DataFrame(W,columns=df_trade_simp.index,index=df_trade_simp.columns)

    # Update
    W_all = pd.DataFrame(np.zeros([len(unique_ISO),len(unique_ISO)]), columns=unique_ISO, index=unique_ISO) # should be 190
    W_all.update(W_df)

    # Save also trade matrix complete
    df_trade_all = pd.DataFrame(np.zeros([len(unique_ISO),len(unique_ISO)]), columns=unique_ISO, index=unique_ISO) # should be 190
    df_trade_all.update(matrix_trade)
    df_trade_all.rename_axis('destin_country_ISO', axis='columns',inplace=True)
    df_trade_all.rename_axis('origin_country_ISO', axis='index',inplace=True)

    return W_all, df_trade_all


In [10]:
# Load data
data_trade_og = pd.read_pickle('../Data/Data_trade_filt_production.pkl')
data_prod_og = pd.read_pickle('../Data/Data_prod_filt_production.pkl')

print(len(data_trade_og.item.unique()))

#Extract list of all countries 
countries_iso = data_trade_og[['origin_country','origin_country_ISO']].rename(columns={'origin_country':'country','origin_country_ISO':'ISO'})
countries_iso = pd.concat([countries_iso,data_trade_og[['destin_country','destin_country_ISO']].rename(columns={'destin_country':'country','destin_country_ISO':'ISO'})])
countries_iso = countries_iso.drop_duplicates().sort_values(by='country')

countries_iso

216


,country,ISO
96120,Afghanistan,AF
118990,Albania,AL
14665,Algeria,DZ
195495,Angola,AO
582661,Antigua and Barbuda,AG
...,...,...
21300329,Wake Island,UM-79
82137,Yemen,YE
585995,Yugoslav SFR,YU
133261,Zambia,ZM


In [4]:
# Take one product & year to work with:
data_trade = data_trade_og.loc[(data_trade_og.item_code ==15) & (data_trade_og.year ==2019)].reset_index(drop=True)
data_prod = data_prod_og.loc[(data_prod_og.item_code ==15) & (data_prod_og.year ==2019) ].reset_index(drop=True)

data_prod = pd.merge(data_prod, countries_iso, left_on= 'area',right_on='country',how='left')
data_prod

unique_ISO = list(sorted(set(data_trade.origin_country_ISO.unique()).union(data_trade.destin_country_ISO.unique()).union(data_prod.ISO.unique())))

# Simplify datasets to work better: 
data_prod= data_prod.loc[:,['ISO','value']]
data_trade_filt = data_trade.loc[:,['origin_country_ISO','destin_country_ISO','value']].reset_index(drop=True)
data_trade_filt = data_trade_filt.loc[data_trade_filt.value>0,:]

In [5]:
# Estimate preferences Trade
W, df_trade_all = Trade_preferences(data_trade_filt, unique_ISO)
df_trade_all


destin_country_ISO,AE,AF,AG,AL,AM,AO,AR,AT,AU,AZ,...,UZ,VC,VE,VN,VU,WS,YE,ZA,ZM,ZW
origin_country_ISO,,,,,,,,,,,,,,,,,,,,,
AE,0.00,0.4,0.0,0.0,0.76,4.00,0.0,0.0,0.08,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,2.15,0.00,0.05,0.00
AF,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00
AG,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00
AL,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00
AM,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WS,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00
YE,13.89,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00
ZA,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.04,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,22270.02,45254.41


In [6]:
   
def Estimate_demand (trade_in, data_prod):
    # Estimate trade from previous timestep:
    in_group = trade_in.groupby(by = 'destin_country_ISO').value.sum()#.round(2) #imports 
    prod_group = data_prod.groupby(by='ISO').value.sum() #.round(2) # production
    ex_group = trade_in.groupby(by = 'origin_country_ISO').value.sum()#.round(2)

    # Define aggregate trade per country: 
    available_product = pd.merge(prod_group, in_group, left_index=True, right_index=True,how='outer',suffixes=['_prod','_import']).fillna(0)
    available_product = pd.merge(available_product, ex_group, left_index=True, right_index=True,how='outer').fillna(0)# exports
    available_product.columns= ['production','imports','exports']

    # Define demand
    demand = (available_product['production'] + available_product['imports'] - available_product['exports']).round(6)
    return demand

# Define shock 
shocks = pd.DataFrame([['IN',1, 1e8]],columns=['ISO','t_shock','magnitude'])

# Test iterations simulation
demand = Estimate_demand(data_trade_filt, data_prod) 


In [14]:

def Iteration_simulation(trade_in, data_prod, shocks, it):  
    # Apply shock to production
    shock_t = shocks.loc[shocks.t_shock== it,:]
    #print(shock_t)

    if len(shock_t) > 0:
        print('in_shock')
        shock_t.index= shocks.ISO
        data_prod_new = data_prod.copy()
        data_prod_new.index = data_prod_new.ISO
        data_prod_new.loc[shock_t.ISO,'value'] = data_prod_new.loc[shock_t.ISO,'value'] - shock_t.loc[:,'magnitude']
        data_prod_new.reset_index(drop=True,inplace=True)
    else: 
        data_prod_new = data_prod.copy()

    # Estimate trade from previous timestep:
    in_group = trade_in.groupby(by = 'destin_country_ISO').value.sum()#.round(2) #imports 
    prod_group = data_prod_new.groupby(by='ISO').value.sum()#.round(2) # production
    ex_group = trade_in.groupby(by = 'origin_country_ISO').value.sum()#.round(2)

    # Define aggregate trade per country: 
    available_product = pd.merge(prod_group, in_group, left_index=True, right_index=True,how='outer',suffixes=['_prod','_import']).fillna(0)
    available_product = pd.merge(available_product, ex_group, left_index=True, right_index=True,how='outer').fillna(0)# exports
    available_product.columns= ['production','imports','exports']

    # Define estimate of drop in product available: 
    available_product['avail_prod'] = (available_product['production'] + available_product['imports'])#.round(6)
    available_product['expected_dd']  = (demand - available_product['avail_prod'] + available_product['exports'])#.round(6)

    # replace isclose by 0: 
    #available_product.loc[np.isclose(available_product['expected_dd'],0.0, atol=1e-9),'expected_dd']=0

    # Estimate change in total exports of the country 
    export_reduction =  pd.concat([available_product['exports'], available_product['expected_dd']],axis=1).min(axis=1)
    available_product['export_reduction'] = export_reduction
    
    available_product['final_dd']  = (available_product['expected_dd']- available_product['export_reduction'])

    available_product.loc[:,'exports_new'] = (available_product['exports'] - export_reduction)
    available_product = available_product.round(6)
    print('ex: ',sum(available_product['exports']),' dd: ',sum(available_product.loc['expected_dd']),' exp_new: ',sum(available_product['exports_new']),' final_dd: ', sum(available_product['final_dd']))
    
    # Propagate the reduction on trade to neighbours:
    next_exports = np.diag(available_product['exports_new'])
    df_trade_new =np.matmul(next_exports, W)
    df_trade_new.index = available_product.index

    # Transform data to long format
    data_trade_new= df_trade_new.stack().reset_index()
    data_trade_new.columns= ['origin_country_ISO','destin_country_ISO','value']

    return data_trade_new, available_product
 
max_t= 350
data_trade = data_trade_filt.copy()
shocks = pd.DataFrame([['IN',1, 1e8]],columns=['ISO','t_shock','magnitude'])

for it in range(1,2):
    data_trade, available_product = Iteration_simulation(data_trade, data_prod, shocks, it) 

in_shock
ex:  185352764.85999998  dd:  100000000.0  exp_new:  185162533.77  final_dd:  99809768.91


In [10]:
available_product.loc['IN',:]

production          1.035962e+08
imports             2.188920e+03
exports             1.902311e+05
avail_prod          1.035984e+08
expected_dd        -1.000000e-06
export_reduction   -1.000000e-06
final_dd            0.000000e+00
exports_new         1.902311e+05
Name: IN, dtype: float64

In [ ]:

# Iteration
#data_trade1, data_prod1, available_product1 = Iteration_simulation(data_trade_filt, data_prod, shocks, 1) 
#data_trade_new2, data_prod_new2,available_product2 = Iteration_simulation(data_trade1, data_prod, shocks, 2) 
data_trade_new3, data_prod_new3,available_product3 = Iteration_simulation(data_trade_new2, data_prod, shocks, 3) 
data_trade_new5, data_prod_new5,available_product5 = Iteration_simulation(data_trade_new3, data_prod, shocks, 4) 



In [ ]:
data_trade_new3, data_prod_new3,available_product3 = Iteration_simulation(data_trade_new2, data_prod, shocks, 3) 

#data_trade_new, data_prod_new, demand = Iteration_simulaiton(data_trade_filt, data_prod, demand, shocks, 1) 


In [ ]:
available_product.loc[np.isclose(available_product['expected_dd'],0.0, atol=1e-9),'expected_dd']=0
available_product

In [ ]:
available_product['expected_dd']

In [ ]:
available_product1['drop_demand']

In [ ]:
available_product2['drop_demand']

In [ ]:
df_trade1.loc['AE',:]

In [ ]:
data_trade_new2

In [ ]:
sum(data_trade_filt.sort_values(by=['origin_country_ISO','destin_country_ISO']).reset_index(drop=True).value == data_trade2.sort_values(by=['origin_country_ISO','destin_country_ISO']).reset_index(drop=True).value)

In [ ]:

# Timestep 0:
in_group = data_trade.groupby(by = 'destin_country_ISO').value.sum()#.round(2) #imports
prod_group = data_prod.groupby(by='ISO').value.sum() #.round(2) # production
ex_group = data_trade.groupby(by = 'origin_country_ISO').value.sum()#.round(2)

available_product = pd.merge(prod_group, in_group, left_index=True, right_index=True,how='outer',suffixes=['_prod','_import']).fillna(0)
available_product = pd.merge(available_product,ex_group,left_index=True, right_index=True,how='outer').fillna(0)# exports
available_product.columns= ['production','imports','exports']

available_product['avail_prod']= (available_product['production']+available_product['imports']).round(6)
av_product_old = available_product.copy()

# Find demand (assumed to be real dc):
available_product['demand'] = (available_product['avail_prod'] - available_product['exports']).round(6)

# Drop in demand
available_product['drop_demand']  = np.abs(available_product['demand'] - available_product['avail_prod'] + available_product['exports']).round(6)

demand_diag= pd.DataFrame(np.diag(available_product['demand']),columns= available_product.index,index= available_product.index)

av_product_old = available_product.copy()
available_product

In [ ]:
shocks = pd.DataFrame([['ZA',1,1e6],['AF',1,1e6]],columns=['ISO','t_shock','magnitude'])
#shocks.loc[shocks.t_shock==it,:]
shock_t = shocks.loc[shocks.t_shock==1,:]
shock_t.index= shocks.ISO
if len(shock_t) > 0: 
    data_prod_new = data_prod.copy()
    data_prod_new.index = data_prod.ISO
    data_prod_new.loc[shocks.ISO,'value'] = data_prod_new.loc[shocks.ISO,'value'] - shock_t.loc[:,'magnitude']

data_prod_new.reset_index(drop=True)


In [ ]:
    data_prod_new

In [ ]:
shock_t['magnitude']

In [ ]:
# Iteration 1: 
# Update available product t=1: 
prod_modified = data_prod.copy()
prod_modified.loc[prod_modified.ISO=='ZA','value'] -= 1e6

# Update available product
available_product['production'] = prod_modified.groupby('ISO').value.sum()
available_product['production']=available_product['production'].fillna(0)

available_product['avail_prod'] = (available_product['production'] +available_product['imports']).round(6)
available_product['drop_demand']  = (available_product['demand'] - available_product['avail_prod'] + available_product['exports']).round(6)

available_product.loc[:,'exports'] = av_product_old['exports'] - pd.concat([av_product_old['exports'], available_product['drop_demand']],axis=1).min(axis=1)

# Spread lack demand: reduce exports proportionally 
df_trade_new =np.matmul(np.diag(available_product['exports']), W)
df_trade_new.index = available_product.index

# New imports

In [ ]:

# Update available product
available_product['production'] = prod_modified.groupby('ISO').value.sum()
available_product['production']=available_product['production'].fillna(0)

available_product['avail_prod'] = available_product['avail_prod']= (available_product['production'] +available_product['imports']).round(6)
available_product['drop_demand']  = np.abs(available_product['demand'] - available_product['avail_prod'] + available_product['exports']).round(6)


# Find drop demand
dd  = np.abs(available_product['demand'] - available_product['avail_prod'] + available_product['exports']).round(6)


# OLD CODE

In [ ]:

row_sum= matrix_trade.sum(axis=1) #axis =1 is rownorm
row_sum= row_sum[row_sum>0]

# Remove rows with zero sum 
matrix_trade_simp= matrix_trade.loc[row_sum.index,:] 
df_trade_simp = pd.DataFrame(matrix_trade_simp, columns=matrix_trade_simp.index, index=matrix_trade_simp.columns) 
inv_sum = np.diag(1/row_sum.to_numpy())

# Get matrix multiplication
W = np.matmul(inv_sum,matrix_trade_simp.to_numpy())
W_df = pd.DataFrame(W,columns=df_trade_simp.index,index=df_trade_simp.columns)

# Update
W_all = pd.DataFrame(np.zeros([len(unique_ISO),len(unique_ISO)]), columns=unique_ISO, index=unique_ISO) # should be 190
W_all.update(W_df)
W_all

In [ ]:
data_tr

In [ ]:
# Iteration 0: 
# Find available product (t=0):
def Iteration_estimate(data_trade, data_prod, av_product_old):

    # Find group 
    in_group = data_trade.groupby(by = 'destin_country_ISO').value.sum()#.round(2) #imports
    prod_group = data_prod.groupby(by='ISO').value.sum() #.round(2) # production
    ex_group = data_trade.groupby(by = 'origin_country_ISO').value.sum()#.round(2)

    # Define available product
    available_product = pd.merge(prod_group, in_group, left_index=True, right_index=True,how='outer',suffixes=['_prod','_import']).fillna(0)
    available_product = pd.merge(available_product,ex_group,left_index=True, right_index=True,how='outer').fillna(0)# exports
    available_product.columns= ['production','imports','exports']

    av_product_old['avail_prod']= (av_product_old['production']+av_product_old['imports']).round(6)

    # Update drop in demand
    available_product['drop_demand']  = np.abs(av_product_old['demand'] - available_product['avail_prod'] + av_product_old['exports']).round(6)

    av_product_old = available_product.copy()

    # Update trade: 
    
    return 





# Scenario 0
in_group = data_trade.groupby(by = 'destin_country_ISO').value.sum()#.round(2) #imports
prod_group = data_prod.groupby(by='ISO').value.sum() #.round(2) # production
ex_group = data_trade.groupby(by = 'origin_country_ISO').value.sum()#.round(2)

available_product = pd.merge(prod_group, in_group, left_index=True, right_index=True,how='outer',suffixes=['_prod','_import']).fillna(0)
available_product = pd.merge(available_product,ex_group,left_index=True, right_index=True,how='outer').fillna(0)# exports
available_product.columns= ['production','imports','exports']

available_product['avail_prod']= (available_product['production']+available_product['imports']).round(6)
av_product_old = available_product.copy()

# Find demand (assumed to be real dc):
available_product['demand'] = (available_product['avail_prod'] - available_product['exports']).round(6)

# Drop in demand
available_product['drop_demand']  = np.abs(available_product['demand'] - available_product['avail_prod'] + available_product['exports']).round(6)

demand_diag= pd.DataFrame(np.diag(available_product['demand']),columns= available_product.index,index= available_product.index)

av_product_old = available_product.copy()

In [ ]:
available_product['drop_demand']

In [ ]:

# Iteration 1: 
# Update available product t=1: 
prod_modified = data_prod.copy()
prod_modified.loc[prod_modified.ISO=='ZA','value'] -= 1e6

# Update available product
available_product['production'] = prod_modified.groupby('ISO').value.sum()
available_product['production']=available_product['production'].fillna(0)

available_product['avail_prod'] = available_product['avail_prod']= (available_product['production'] +available_product['imports']).round(6)
available_product['drop_demand']  = np.abs(available_product['demand'] - available_product['avail_prod'] + available_product['exports']).round(6)


# Find drop demand
dd  = np.abs(available_product['demand'] - available_product['avail_prod'] + available_product['exports']).round(6)

# Countries affected: 
np.diag(dd)* W_df




In [ ]:
available_product

In [ ]:
# Iteration 1: 
# Update available product t=1: 
prod_modified = data_prod.copy()
prod_modified.loc[prod_modified.ISO=='RU','value'] -= 20e6
prod_modified

# test Delta d
dd  = available_product['demand'] - available_product['imports'] - available_product['production'] + ex_group

# Find drop in demand
in_group = data_trade.groupby(by = 'destin_country_ISO').value.sum() #imports
prod_group = prod_modified.groupby(by='ISO').value.sum() # production

available_product = pd.merge(prod_group, in_group, left_index=True, right_index=True,how='outer',suffixes=['_prod','_import']).fillna(0)
available_product['avail_prod']= available_product['value_prod']+available_product['value_import']

delta_available= available_product['avail_prod']- av_product_old['avail_prod']

dd


In [ ]:
# IN-trade sum all years:  
in_trade_tot = data_trade.groupby(by = 'destin_country').value.sum()
production_tot= data_prod.groupby(by='area').value.sum()
out_product_tot =  data_trade.groupby(by='origin_country').value.sum()
all_keys_tot = list(set(in_trade_tot.keys()).union(production_tot.keys()).union(out_product_tot.keys()))
all_keys_tot

In [ ]:

def Available_product(in_trade, production, out_product, key_pair):

    imports = 0
    if key_pair in in_trade.keys() :
        imports =  in_trade[key_pair]
    
    prod = 0
    if key_pair in production.keys() :
        prod =  production[key_pair]

    available_product = imports + prod

    # output 
    exported_product = 0
    if key_pair in out_product.keys() :
        #print('output:', out_product[key_pair])
        exported_product= out_product[key_pair]

    # DOMESTIC CONSUMPTION 
    dom_con = available_product - exported_product

    if type(key_pair) is str:
        return_file =[key_pair,'All',imports, prod, available_product,exported_product,dom_con]
    else:
        return_file =[key_pair[0],key_pair[1],imports, prod, available_product,exported_product,dom_con]

    return return_file

#out_data = [Available_product(in_trade, production, out_product, key_pair) for key_pair in all_keys]
#data_check = pd.DataFrame(out_data,columns = ['country','year','imports','production','avail_prod','export_product','domest_consum'])

# Check total sum all years: 
out_data_tot = [Available_product(in_trade_tot, production_tot, out_product_tot, key_pair) for key_pair in all_keys_tot]
data_check_tot = pd.DataFrame(out_data_tot,columns = ['country','year','imports','production','avail_prod','export_product','domest_consum'])

In [ ]:
# Focus on the 2019 wheat trade: 
#df1= data_trade.loc[:,['origin_country_ISO','origin_country']].rename(columns={'origin_country_ISO':'ISO','origin_country': 'country'})
#df2= data_trade.loc[:,['destin_country_ISO','destin_country']].rename(columns={'destin_country_ISO':'ISO','destin_country': 'country'})
#all_iso = pd.concat([df1, df2],axis=0, ignore_index=True).drop_duplicates()

# Add domestic consumption to trade_list: 
data_agg = pd.merge(data_check_tot, countries_iso, on = 'country')
prod_country_key = data_agg[['ISO','country']]
prod_country_key 
dom_consumption=data_agg.loc[:,['domest_consum','ISO']]
dom_consumption['destin_country_ISO'] = dom_consumption['ISO']
dom_consumption=dom_consumption.rename(columns={'ISO': 'origin_country_ISO','domest_consum':'value'})


In [ ]:
data_trade

In [ ]:
# TRADE MATRIX
df1= data_trade.loc[:,['origin_country_ISO','origin_country']].rename(columns={'origin_country_ISO':'ISO','origin_country': 'country'})
df2= data_trade.loc[:,['destin_country_ISO','destin_country']].rename(columns={'destin_country_ISO':'ISO','destin_country': 'country'})
all_iso = pd.concat([df1, df2, prod_country_key],axis=0, ignore_index=True).drop_duplicates()

df_trade = data_trade.loc[:,['origin_country_ISO','destin_country_ISO','value']]
#df_trade = pd.concat([df_trade, dom_consumption], ignore_index=True)

matrix_trade = df_trade.pivot(index = 'origin_country_ISO',columns='destin_country_ISO',values='value').fillna(0)
matrix_domestic = np.diag(dom_consumption)


# UPDATE MATRIX 
df_trade_all = pd.DataFrame(np.zeros([len(all_iso),len(all_iso)]), columns=all_iso.ISO, index=all_iso.ISO) # should be 190
df_trade_all.update(matrix_trade)
df_trade_all.rename_axis('destin_country_ISO', axis='columns',inplace=True)
df_trade_all.rename_axis('origin_country_ISO', axis='index',inplace=True)
df_trade_all

#Divide by rowsum: D= Id*(1/rowsum)
row_sum= df_trade_all.sum(axis=1) #axis =1 is rownorm

inv_sum = np.diag(1/row_sum.to_numpy())

#Multiply D*W
W = np.matmul(inv_sum,df_trade_all.to_numpy())

W_df = pd.DataFrame(W,columns=df_trade_all.columns,index=df_trade_all.index)
W_df

# Sort data agg
data_agg.index=data_agg.ISO
data_agg = data_agg.loc[all_iso.ISO,:]

In [ ]:
# update test? 
prod_vec = np.array(data_agg.avail_prod)
np.diag(prod_vec)

redivided_share = np.matmul(np.diag(prod_vec),W)

In [ ]:
# old state
og_state = df_trade_all

# 
produciton_old = data_agg.loc['RU','production']
produciton = produciton_old - 20000000

df_trade_all
production_diff = df_trade_all  


In [ ]:
long_data = df_trade_all.unstack().reset_index()
long_data.columns= ['destin_country_ISO','origin_country_ISO','value']
long_data

# in trade: 
imports =  in_trade[key_pair]


In [ ]:
inputs = long_data.groupby('destin_country_ISO').value.sum()
inputs


In [ ]:
df_trade_all

In [ ]:

def Update_states(old_trade, old_production, out_product, key_pair):
    old_trade = df_trade_all

    imports = 0
    if key_pair in in_trade.keys() :
        imports =  in_trade[key_pair]
    
    prod = 0
    if key_pair in production.keys() :
        prod =  production[key_pair]

    available_product = imports + prod

    # output 
    exported_product = 0
    if key_pair in out_product.keys() :
        #print('output:', out_product[key_pair])
        exported_product= out_product[key_pair]

    # DOMESTIC CONSUMPTION 
    dom_con = available_product - exported_product

    if type(key_pair) is str:
        return_file =[key_pair,'All',imports, prod, available_product,exported_product,dom_con]
    else:
        return_file =[key_pair[0],key_pair[1],imports, prod, available_product,exported_product,dom_con]

    return return_file


out_data = [Available_product(in_trade, production, out_product) for key_pair in all_keys]
data_check = pd.DataFrame(out_data,columns = ['country','year','imports','production','avail_prod','export_product','domest_consum'])


In [ ]:
old_trade = df_trade_all
old_trade

In [ ]:
prod_vec = np.array(data_agg.avail_prod)
np.diag(prod_vec)

redivided_share = np.matmul(np.diag(prod_vec),W)

# 

In [ ]:
product_division =W_df.loc['RU',:] * data_agg.loc['RU','avail_prod']


In [ ]:
pd.dot(prod_dc)

In [ ]:
# Sort data agg to get same order than all_iso
data_agg.index=data_agg.ISO
data_agg = data_agg.loc[all_iso.ISO,:]
prod_vec = np.array(data_agg.avail_prod)
# 
av_cons= W.mul(prod_vec)

#avail_prod = av_cons.sum(axis=1)
av_cons.sum(axis=1)

In [ ]:
data_agg

In [ ]:
W